# Introduction to Julia by Dr. Mohammad Masiur Rahaman (masiurr@iitbbs.ac.in)

## Probelm No. 1: Generate triangular mesh for a rectangle of length L and height H using Gmsh package in Julia

In [ ]:
# input parameters
L = 2;
H = 1;
h = 0.25;

In [ ]:
import Gmsh: gmsh

# Initializing Gmsh (Initialize the Gmsh API)
gmsh.initialize()
# Create a Model (For example, let's create a rectangle of length L and height H)
gmsh.model.add("Rectangle")

# Define points
p1 = gmsh.model.geo.addPoint(0, 0, 0, h)
p2 = gmsh.model.geo.addPoint(L, 0, 0, h)
p3 = gmsh.model.geo.addPoint(L, H, 0, h)
p4 = gmsh.model.geo.addPoint(0, H, 0, h)

# Add lines
l1 = gmsh.model.geo.addLine(p1, p2)
l2 = gmsh.model.geo.addLine(p2, p3)
l3 = gmsh.model.geo.addLine(p3, p4)
l4 = gmsh.model.geo.addLine(p4, p1)

# Create a curve loop and surface
cl = gmsh.model.geo.addCurveLoop([l1, l2, l3, l4])
s = gmsh.model.geo.addPlaneSurface([cl])

# Create physical group for domain
gmsh.model.addPhysicalGroup(2, [s], 10, "Domain")

# Create physical groups for boundaries
gmsh.model.addPhysicalGroup(1, [l1], 1, "Bottom")
gmsh.model.addPhysicalGroup(1, [l2], 2, "Right")
gmsh.model.addPhysicalGroup(1, [l3], 3, "Top")
gmsh.model.addPhysicalGroup(1, [l4], 4, "Left")

# Synchronize and Mesh (Synchronize the CAD model and generate a mesh:)
gmsh.model.geo.synchronize()
gmsh.model.mesh.generate(2)

# Write mesh to file
gmsh.write("rectangle.msh")

#Launch GUI or Finalize (Launch the GUI to visualize the mesh or finalize the Gmsh API:)
gmsh.fltk.run()  # Launch GUI
# or 
gmsh.finalize() # Finalize Gmsh API

### To see the mesh file in paraview 

In [ ]:
using Gridap
using GridapGmsh
model = DiscreteModelFromFile("rectangle.msh")

In [ ]:
writevtk(model,"model")

## Probelm No. 2: Generate rectangular mesh for a rectangle of length L and height H using GridapGmsh package in Julia

In [ ]:
using Gridap
using GridapGmsh

domain = (0,L,0,H)
partition = (10,5)
model = CartesianDiscreteModel(domain,partition)

#Define new boundaries
labels = get_face_labeling(model)
add_tag_from_tags!(labels,"Left",[1,3,7]);
add_tag_from_tags!(labels,"Right",[2,4,8]);
add_tag_from_tags!(labels,"Top",[3,4,6]);
add_tag_from_tags!(labels,"Bottom",[1,2,5]);

In [ ]:
writevtk(model,"modelRectEle")

## Probelm No. 3: Generate triangular mesh for a Brick

In [ ]:
# Define brick dimensions
L, W, H = 2.0, 1.0, 0.5;
h = 0.1;

In [ ]:
using Gmsh

# Initialize Gmsh
gmsh.initialize()

# Create a new model
gmsh.model.add("brick")

# Define corner points
p1 = gmsh.model.geo.addPoint(0, 0, 0, h)
p2 = gmsh.model.geo.addPoint(L, 0, 0, h)
p3 = gmsh.model.geo.addPoint(L, W, 0, h)
p4 = gmsh.model.geo.addPoint(0, W, 0, h)

# Bottom face lines
l1 = gmsh.model.geo.addLine(p1, p2)
l2 = gmsh.model.geo.addLine(p2, p3)
l3 = gmsh.model.geo.addLine(p3, p4)
l4 = gmsh.model.geo.addLine(p4, p1)

# Curve loop and plane surface for bottom
cl = gmsh.model.geo.addCurveLoop([l1, l2, l3, l4])
s = gmsh.model.geo.addPlaneSurface([cl])

# Extrude to create 3D brick
ext = gmsh.model.geo.extrude([(p2, s)], 0, 0, H)

# Synchronize geometry
gmsh.model.geo.synchronize()

# Generate 3D mesh
gmsh.model.mesh.generate(3)

# Save mesh
gmsh.write("brick.msh")

#Launch GUI or Finalize (Launch the GUI to visualize the mesh or finalize the Gmsh API:)
gmsh.fltk.run()  # Launch GUI
# Finalize Gmsh
gmsh.finalize()

In [ ]:
using Gridap
using GridapGmsh
model = DiscreteModelFromFile("brick.msh")

writevtk(model,"model3Dbrick")

## Probelm No. 4: Generate triangular mesh for anular ring

In [2]:
# Define parameters
inner_radius = 1.0
outer_radius = 2.0
h = 0.1

0.1

In [3]:
import Gmsh: gmsh

# Initialize Gmsh
gmsh.initialize()
gmsh.option.setNumber("General.Terminal", 1)

# Create a new model
gmsh.model.add("AnnularRing")

# Define points
p1 = gmsh.model.geo.addPoint(0, 0, 0, h)
p2 = gmsh.model.geo.addPoint(inner_radius, 0, 0, h)
p3 = gmsh.model.geo.addPoint(-inner_radius, 0, 0 ,h)

p4 = gmsh.model.geo.addPoint(outer_radius, 0, 0, h)
p5 = gmsh.model.geo.addPoint(-outer_radius, 0, 0, h)

# Create circle arcs
c1 = gmsh.model.geo.addCircleArc(p2, p1, p3)
c2 = gmsh.model.geo.addCircleArc(p3, p1, p2)

c3 = gmsh.model.geo.addCircleArc(p4, p1, p5)
c4 = gmsh.model.geo.addCircleArc(p5, p1, p4)

# Create curve loops
inner_curve_loop = gmsh.model.geo.addCurveLoop([c1, c2])
outer_curve_loop = gmsh.model.geo.addCurveLoop([c3, c4])

# Create plane surface
surface = gmsh.model.geo.addPlaneSurface([outer_curve_loop, -inner_curve_loop])

# Synchronize the geometry
gmsh.model.geo.synchronize()

# Generate the mesh
gmsh.model.mesh.generate(2)  # 2D mesh

#Set Element Order (Set the order of the mesh elements:)

element_order = 2
gmsh.model.mesh.setOrder(element_order)

# Mesh and save
gmsh.write("annular_ring.msh")

#Launch GUI or Finalize (Launch the GUI to visualize the mesh or finalize the Gmsh API:)
gmsh.fltk.run()  # Launch GUI

# Finalize Gmsh
gmsh.finalize()

Info    : Meshing 1D...
Info    : [  0%] Meshing curve 1 (Circle)
Info    : [ 30%] Meshing curve 2 (Circle)
Info    : [ 60%] Meshing curve 3 (Circle)
Info    : [ 80%] Meshing curve 4 (Circle)
Info    : Done meshing 1D (Wall 0.000828981s, CPU 0s)
Info    : Meshing 2D...
Info    : Meshing surface 1 (Plane, Frontal-Delaunay)
Info    : Done meshing 2D (Wall 0.0293519s, CPU 0.03125s)
Info    : 1249 nodes 2501 elements
Info    : Meshing order 2 (curvilinear on)...
Info    : [  0%] Meshing curve 1 order 2
Info    : [ 30%] Meshing curve 2 order 2
Info    : [ 50%] Meshing curve 3 order 2
Info    : [ 70%] Meshing curve 4 order 2
Info    : [ 90%] Meshing surface 1 order 2
Info    : Surface mesh: worst distortion = 0.954047 (0 elements in ]0, 0.2]); worst gamma = 0.71347
Info    : Done meshing order 2 (Wall 0.00751185s, CPU 0s)
Info    : Writing 'annular_ring.msh'...
Info    : Done writing 'annular_ring.msh'
-------------------------------------------------------
Version       : 4.13.1
License    

In [4]:
using Gridap
using GridapGmsh
model = DiscreteModelFromFile("annular_ring.msh")

writevtk(model,"modelAnnular_ring")

Info    : Reading 'annular_ring.msh'...
Info    : 10 entities
Info    : 4803 nodes
Info    : 2501 elements
Info    : Done reading 'annular_ring.msh'


LoadError: BoundsError: attempt to access 1249-element Vector{Int32} at index [0]

## Probelm No. 5: Notched Plate with Hole

In [ ]:
const L = 65      #Length
const H = 120      #Height
const Lh = 10     #notch length
const ls = 0.45
const CP = 55      #center height  
const CH = H/40    #Crack height
const Lu = 28.5      #offset of hole from right end
const Ld = 51      #offset of hole from bottom end
const hfc = ls/50
const hf = ls/5   #finer mesh size
const h = 100*hf     #coarser mesh size
const di = 20      # Diameter of hole
const FMR = 7*ls  

In [ ]:
import Gmsh: gmsh

gmsh.initialize()
gmsh.option.setNumber("General.Terminal", 1)
p1 = gmsh.model.geo.addPoint(0.0, 0.0, 0.0, h)   
p2 = gmsh.model.geo.addPoint(L, 0, 0.0, h) 
p3 = gmsh.model.geo.addPoint(L, H, 0.0, h)
p4 = gmsh.model.geo.addPoint(0, H, 0.0, h)
p5 = gmsh.model.geo.addPoint(0, H-CP, 0.0, h)
p6 = gmsh.model.geo.addPoint(Lh, H-CP, 0.0, hfc)
p7 = gmsh.model.geo.addPoint(Lh, H-CP-0.5*CH, 0.0, hfc)
p8 = gmsh.model.geo.addPoint(0, H-CP-0.5*CH, 0.0, h)

p9 = gmsh.model.geo.addPoint(L-Lu-0.5*di, Ld, 0.0, h)
p10 = gmsh.model.geo.addPoint(L-Lu, Ld, 0.0, h)
p11 = gmsh.model.geo.addPoint(L-Lu+0.5*di, Ld, 0.0, h)

p12 = gmsh.model.geo.addPoint(L-Lu, Ld+0.5*di, 0.0, h)

l1 = gmsh.model.geo.addLine(p1, p2)
l2 = gmsh.model.geo.addLine(p2, p3)
l3 = gmsh.model.geo.addLine(p3, p4)
l4 = gmsh.model.geo.addLine(p4, p5)
l5 = gmsh.model.geo.addLine(p5, p6)
l6 = gmsh.model.geo.addLine(p6, p7)
l7 = gmsh.model.geo.addLine(p7, p8)
l8 = gmsh.model.geo.addLine(p8, p1)

l9 = gmsh.model.geo.addLine(p7, p9)
l10 = gmsh.model.geo.addLine(p7, p12)

c1 = gmsh.model.geo.addCircleArc(p9,p10,p11)
c2 = gmsh.model.geo.addCircleArc(p11,p10,p9)

cl1 = gmsh.model.geo.addCurveLoop([l1,l2,l3,l4,l5,l6,l7,l8])
cl2 = gmsh.model.geo.addCurveLoop([c1,c2])

ps1 = gmsh.model.geo.addPlaneSurface([cl1,-cl2])

pg1 = gmsh.model.addPhysicalGroup(2, [ps1])
pg2 = gmsh.model.addPhysicalGroup(1, [l1])
pg3 = gmsh.model.addPhysicalGroup(1, [l3])

gmsh.model.setPhysicalName(2, pg1, "Domain")
gmsh.model.setPhysicalName(1, pg2, "DirichletBot")
gmsh.model.setPhysicalName(1, pg3, "DirichletTop")


gmsh.model.mesh.field.add("Box", 11)
gmsh.model.mesh.field.setNumber(11, "VIn", hf)
gmsh.model.mesh.field.setNumber(11, "VOut", h)
gmsh.model.mesh.field.setNumber(11, "XMin", L-Lu+0.45*di)
gmsh.model.mesh.field.setNumber(11, "XMax", L)
gmsh.model.mesh.field.setNumber(11, "YMin", Ld-FMR)
gmsh.model.mesh.field.setNumber(11, "YMax", Ld+FMR)

gmsh.model.mesh.field.add("Distance", 1)
gmsh.model.mesh.field.setNumbers(1, "EdgesList", [l10])

gmsh.model.mesh.field.add("Threshold", 2)
gmsh.model.mesh.field.setNumber(2, "IField", 1)
gmsh.model.mesh.field.setNumber(2, "LcMin", hf)
gmsh.model.mesh.field.setNumber(2, "LcMax", h)
gmsh.model.mesh.field.setNumber(2, "DistMin", FMR)
gmsh.model.mesh.field.setNumber(2, "DistMax", 0.50*H)

gmsh.model.mesh.field.add("Min", 7)
gmsh.model.mesh.field.setNumbers(7, "FieldsList", [11,2])

gmsh.model.mesh.field.setAsBackgroundMesh(7)

gmsh.model.geo.synchronize()
gmsh.model.mesh.generate(2)
gmsh.write("NotchedPlateWithHole.msh")

#Launch GUI or Finalize (Launch the GUI to visualize the mesh or finalize the Gmsh API:)
gmsh.fltk.run()  # Launch GUI
gmsh.finalize()

In [ ]:
using Gridap
using GridapGmsh
model = DiscreteModelFromFile("NotchedPlateWithHole.msh")

writevtk(model,"modelNotchedPlateWithHole")